In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
url = "https://business-mail.jp/example"
page = requests.get(url)
soup = BeautifulSoup(page.text, "html")

links = soup.find_all("a")
wants = [
    "https://business-mail.jp/cat_example/outside/",
    "https://business-mail.jp/cat_example/office/",
    "https://business-mail.jp/tag_example/"
]
filtered_links = [a['href'] for a in soup.find_all('a') if any(a['href'].startswith(y) for y in wants)]

data = []

for link in filtered_links:
    # get current topic page
    topic_page = requests.get(link)
    topic_soup = BeautifulSoup(topic_page.text, "html")

    # get all example links from current topic
    topic_example_links = [a["href"] for a in topic_soup.find_all("a") \
        if a["href"].startswith("https://business-mail.jp/example/")]
    topic_example_links = [x for x in topic_example_links if x != "https://business-mail.jp/example/"]

    # get title and text from each example
    for example_link in topic_example_links:
        example_page = requests.get(example_link)
        example_soup = BeautifulSoup(example_page.text, "html.parser")
        example_title = example_soup.find_all("h1")[0].get_text()
        example_email = [p.get_text(separator="\n") for p in example_soup.find_all('p')][0]
        data.append([example_link, example_title, example_email])

In [56]:
# clean up and save as csv
columns = ["link", "title", "email"]
df = pd.DataFrame(data, columns=columns)
df = df.drop_duplicates("email").reset_index().drop("index", axis=1)

def truncate_signature(text):
    index = text.find("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    if index != -1:
        return text[:index]
    return text

df["email_no_sig"] = df["email"].apply(lambda x: truncate_signature(x))
df.to_csv("japanese_business_email_examples.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   link          161 non-null    object
 1   title         161 non-null    object
 2   email         161 non-null    object
 3   email_no_sig  161 non-null    object
dtypes: object(4)
memory usage: 5.2+ KB


In [53]:
df = pd.read_csv("japanese_business_email_examples.csv")
df.columns

Index(['Unnamed: 0', 'link', 'title', 'email', 'email_no_sig'], dtype='object')

In [37]:
import re

# Cleaning helper methods

def remove_japanese_and_chinese_characters(s):
    # Unicode ranges for Japanese and Chinese characters
    japanese_and_chinese_ranges = (
        '\u3040-\u309F',  # Hiragana
        '\u30A0-\u30FF',  # Katakana
        '\u4E00-\u9FFF',  # Common Kanji
        '\uF900-\uFAFF',  # Rare Kanji
        # '\u3400-\u4DBF',  # CJK Unified Ideographs Extension A
        # '\u20000-\u2A6DF',  # CJK Unified Ideographs Extension B
        # '\u2A700-\u2B73F',  # CJK Unified Ideographs Extension C
        # '\u2B740-\u2B81F',  # CJK Unified Ideographs Extension D
        # '\u2B820-\u2CEAF',  # CJK Unified Ideographs Extension E
        # '\u2CEB0-\u2EBEF',  # CJK Unified Ideographs Extension F
        '\u3000-\u303F',  # CJK Symbols and Punctuation
    )

    # Create a regex pattern to match these ranges
    pattern = '[' + ''.join(japanese_and_chinese_ranges) + ']'

    # Replace matched characters with an empty string
    return set(re.sub(pattern, '', s))

def remove_extra_delineations(email_body):
    # Replace placeholder symbols
    email_body = email_body.replace('●●●●', '{company_name}')

    # Remove excessive line breaks and normalize spacing
    email_body = re.sub(r'\n\s*\n', '\n', email_body.strip())

    return email_body

In [40]:
df

,Unnamed: 0,link,title,email,email_no_sig
0,0,https://business-mail.jp/example/11047,メールマガジンの発行を知らせるご挨拶メール,株式会社●●●●\n\r\n●●●●様\n\n\n\r\n大変ご無沙汰しております。\n\r...,株式会社●●●●\n\r\n●●●●様\n\n\n\r\n大変ご無沙汰しております。\n\r...
1,1,https://business-mail.jp/example/11011,セミナーのキャンセル手続き完了を知らせるメールの文例,株式会社●●●●\n\r\n●●●●様\n\n\n\r\nお世話になっております。\n\r\...,株式会社●●●●\n\r\n●●●●様\n\n\n\r\nお世話になっております。\n\r\...
2,2,https://business-mail.jp/example/1254,住居表示の変更をお知らせするメールの文例,株式会社●●●●\n\r\n●●●●様\n\n\n\r\nお世話になっております。\n\r\...,株式会社●●●●\n\r\n●●●●様\n\n\n\r\nお世話になっております。\n\r\...
3,3,https://business-mail.jp/example/1252,商品価格の改定をお知らせするメールの文例,株式会社●●●●\n\r\n●●●●様\n\n\n\r\nお世話になっております。\n\r\...,株式会社●●●●\n\r\n●●●●様\n\n\n\r\nお世話になっております。\n\r\...
4,4,https://business-mail.jp/example/1251,営業時間の変更をお知らせするメールの文例,株式会社○○\n\r\n◇◇◇◇様\n\n\n\r\nいつもお世話になっております。\n\r...,株式会社○○\n\r\n◇◇◇◇様\n\n\n\r\nいつもお世話になっております。\n\r...
...,...,...,...,...,...
156,156,https://business-mail.jp/example/478,打ち合わせ時間の変更をお願いするメールの文例,関係各位\n\n\n\n\n\r\nお疲れ様です。○○です。\n\n\n\n\n\r\n明日...,関係各位\n\n\n\n\n\r\nお疲れ様です。○○です。\n\n\n\n\n\r\n明日...
157,157,https://business-mail.jp/example/476,プレゼン資料の作成を依頼するメールの文例,□□さん\n\n\n\r\nお疲れ様です。○○です。\n\n\n\n\n\r\n先ほどの会議...,□□さん\n\n\n\r\nお疲れ様です。○○です。\n\n\n\n\n\r\n先ほどの会議...
158,158,https://business-mail.jp/example/475,システム不具合の調査を依頼するメールの文例,システム部\n\r\n鈴木さん\n\n\n\r\nお疲れ様です。営業部の山田です。\n\n\...,システム部\n\r\n鈴木さん\n\n\n\r\nお疲れ様です。営業部の山田です。\n\n\...
159,159,https://business-mail.jp/example/745,人員増強の相談をするメールの文例,○○課長\n\n\n\r\nお疲れ様です、○○です。\n\n\n\r\n現在進行している●●...,○○課長\n\n\n\r\nお疲れ様です、○○です。\n\n\n\r\n現在進行している●●...
